# DSC Coding Challenge

In this challenge, we design and implement a feature selection process that automatically discovers the best feature to use in a linear model given a training set of labeled feature vectors.

## The Dataset

We start   with   a   dataset   consisting   of   100,000   vectors   with   1001   dimensions.   The   values   in   each dimension   are   floating-point   numbers.   The   first   dimension   is   the  l   label    and   the   subsequent   1000 dimensions   are    features .   The   label   and   features   are   all   real-valued    random   variables    for   which the   values   in   each   vector   are    random   variants .   The   label   can   be   modeled   as   a   linear combination   of   a   subset   of   features;   i.e.,   there   is   a   simple    linear   model    for   the   label   in   terms   of certain   features.   Most   features   are    not    related   to   the   label.

## Feature Selection
Our   goal   is   to   design   and   implement   a    feature   selection   process    that   discovers   the   best features   to   use   in   a   linear   model   of   the   label   in   the   dataset   described   above.   Assume   that   a separate   process   will   use   Linear   Regression   to   produce   the   final   model   in   terms   of   the   features chosen   by   your   process.   Your   process   only   needs   to   specify   the   features   on   which   to   perform the   final   model   selection.

## The Linear Regression Model
After reducing the feature set drastically (by only considering features that are "statistically significant" with p values < 10^{-3}), we construct simple linear regression models to illustrate the improvment in model-training time.

### Import necessary libraries

In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_regression, SelectKBest, chi2, RFE
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

### Import the data

In [26]:
data = pd.read_csv("../data/linear_regression_challenge.csv",
                   sep="|",
                   header=None)

print("Dimensions of the data: ", data.shape)

Dimensions of the data:  (100000, 1001)


### Split the Data Into Feature-Extraction, Training and Validation Sets

In [27]:
# shuffle the data
data = data.sample(frac=1.0, random_state=42)

# data to learn significant features
feat_data = data[:]
X_feat = feat_data.drop(0, axis=1)
y_feat = feat_data[0]

# data to train a linear regression model
train_data = data[: 90000]
X_train = train_data.drop(0, axis=1)
y_train = train_data[0]

# data to test the linear regression model
valid_data = data[90000:]
X_valid = valid_data.drop(0, axis=1)
y_valid = valid_data[0]

print("Dimensions of feature-learning data: ", feat_data.shape)
print("Dimensions of model-training data: ", train_data.shape)
print("Dimensions of model-validation data: ", valid_data.shape)

Dimensions of feature-learning data:  (100000, 1001)
Dimensions of model-training data:  (90000, 1001)
Dimensions of model-validation data:  (10000, 1001)


### Normalize the Features
The features are NOT commensurate.  Therefore, we will normalize them using scikit-learn's "MinMaxScaler" which will scale all features to be in a range between 0 and 1.  The transformation is given by:

X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

X_scaled = X_std * (max - min) + min

In [28]:
min_max_scaler = MinMaxScaler()

X_feat_scaled = pd.DataFrame(min_max_scaler.fit_transform(X_feat))
X_train_scaled = pd.DataFrame(min_max_scaler.transform(X_train))
X_valid_scaled = pd.DataFrame(min_max_scaler.transform(X_valid))

print("The dimensions of the scaled feature set: ", X_feat_scaled.shape)
X_feat_scaled.head()

The dimensions of the scaled feature set:  (100000, 1000)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.583405,0.681160,0.952492,0.000000,0.765276,0.000000,0.072089,0.000000,0.204099,0.184882,...,0.097690,0.492290,0.000000,0.668970,0.570888,0.000000,0.592841,0.228362,0.631634,0.463578
1,0.674569,0.966525,0.425462,0.000000,0.879005,0.755205,0.519232,0.676201,0.218691,0.749504,...,0.514684,0.041401,0.740817,0.336196,0.755942,0.884311,0.824636,0.360692,0.000000,0.766493
2,0.599700,0.000000,0.450182,0.877262,0.639430,0.000000,0.901275,0.688615,0.192681,0.095179,...,0.092531,0.402710,0.503532,0.609030,0.000000,0.000000,0.990625,0.906956,0.594805,0.838676
3,0.415988,0.374553,0.000000,0.969008,0.742408,0.503840,0.804336,0.446704,0.470925,0.395633,...,0.425341,0.615899,0.551023,0.732308,0.000000,0.654378,0.990732,0.877018,0.846179,0.798890
4,0.849916,0.552855,0.953682,0.344811,0.846622,0.320495,0.201693,0.861966,0.173841,0.935148,...,0.723803,0.385278,0.000000,0.450559,0.368364,0.955237,0.756991,0.568739,0.000000,0.179280


### Univariate Feature Filtering
Assessing individual features won't scale as we increase the number of our features.  Let's try it anyway just so we can see which individual features are important and get a sense of how long it takes.

To do this, we can make use of Scikit-Learn's "f_regression" method.  It returns the F-score as well as the "p-value" of the "F-score".  We'll use the standard that any feature with a p-value < 10^{-3} is "significant".

In [29]:
%%time
f_values = f_regression(X_feat_scaled, y_feat)

significant_count = 0
significant_idx = []
for i in range(f_values[0].shape[0]):
    if f_values[1][i] < 1e-3:
        significant_count += 1
        significant_idx.append(i)
        
significant_idx = np.array(significant_idx)
     

print("Selected Features: ", list(significant_idx))
print("There are {} significant features.".format(significant_count))
print()

Selected Features:  [11, 332, 334, 403, 701]
There are 5 significant features.

CPU times: user 349 ms, sys: 530 ms, total: 879 ms
Wall time: 862 ms


### Scikit-Learn has an extension of f_regression called "SelectKBest" which we can use to select (using F scores) the "K" best features.

In [30]:
%%time
selector = SelectKBest(f_regression, k=significant_count)
selector.fit_transform(X_feat_scaled, y_feat)

idxs_selected = selector.get_support(indices=True)

idxs_intersect = list(np.intersect1d(list(significant_idx), list(idxs_selected)))

print("Selected Features: ", list(idxs_selected))
print()
print("Intersection with univariate approach: ", idxs_intersect)
print("Percentage overlap: " + str(len(idxs_intersect) / len(idxs_selected) * 100.) + "%")
print()

Selected Features:  [11, 332, 334, 403, 701]

Intersection with univariate approach:  [11, 332, 334, 403, 701]
Percentage overlap: 100.0%

CPU times: user 319 ms, sys: 5.56 ms, total: 325 ms
Wall time: 307 ms


### Testing for Outliers

In [31]:
%%time
# use numpy to calculate mean and variance (covariance matrix)
#def outlier_detect(x):
#    mu = np.mean(x, axis=0)
#    sigma = np.cov(x)
#    return np.abs(x-x.mean())<= 3 * x.std()

#X_scaled_nooutliers = X_scaled[X_scaled.apply(outlier_detect, axis=1)]
    
#print(X_scaled_nooutliers.shape())

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


### Use "Recursive Feature Elimination" to select features

In [32]:
%%time
estimator = LinearRegression()
num_features = significant_count
remove_features_step = 0.5

selector = RFE(estimator,
               n_features_to_select=num_features, 
               step=remove_features_step)

selector = selector.fit(X_feat_scaled, y_feat)

selected = selector.ranking_ == 1

idxs_selected_rfe = selected.nonzero()[0]

idxs_intersect_rfe1 = np.intersect1d(list(significant_idx), list(idxs_selected_rfe))
idxs_intersect_rfe2 = np.intersect1d(list(idxs_selected), list(idxs_selected_rfe))

print("Selected features: ", list(idxs_selected_rfe))
print()
print("Intersection with univariate approach: ", idxs_intersect_rfe1)
print("Percentage overlap with univariate approach: " + str(np.round(len(idxs_intersect_rfe1) / len(idxs_selected) * 100., 1)) + "%")
print()
print("Intersection with SelectKBest approach: ", idxs_intersect_rfe2)
print("Percentage overlap with SelectKBest approach: " + str(np.round(len(idxs_intersect_rfe2) / len(idxs_selected) * 100., 1)) + "%")
print()

Selected features:  [11, 332, 334, 403, 701]

Intersection with univariate approach:  [ 11 332 334 403 701]
Percentage overlap with univariate approach: 100.0%

Intersection with SelectKBest approach:  [ 11 332 334 403 701]
Percentage overlap with SelectKBest approach: 100.0%

CPU times: user 23 s, sys: 4.42 s, total: 27.4 s
Wall time: 19 s


## Build a Linear Regression Model

Train and validate a linear regression model using both the full set of 1000 features and then the reduced set to compare and test the feature selection tool.

### First, build a model using the full feature set

In [33]:
%%time
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
print("Coefficient of determination: R^2 = ", lr.score(X_valid_scaled, y_valid))
print()

Coefficient of determination: R^2 =  0.998963814358
CPU times: user 15.4 s, sys: 2.15 s, total: 17.5 s
Wall time: 11.5 s


### Next, build a model using the greatly-reduced feature set

In [35]:
%%time
X_train_reduced = X_train_scaled[list(idxs_selected_rfe)]
X_valid_reduced = X_valid_scaled[list(idxs_selected_rfe)]

lr = LinearRegression()
lr.fit(X_train_reduced, y_train)
print("Coefficient of determination: R^2 = ", lr.score(X_valid_reduced, y_valid))
print()

Coefficient of determination: R^2 =  0.998811993654
CPU times: user 12.2 ms, sys: 4.49 ms, total: 16.6 ms
Wall time: 16.7 ms


## Final Comments

We see that by reducing the number of features the speed of our linear regression model is nearly 1000 times faster (while still maintaining the high performance!).